<a href="https://colab.research.google.com/github/Kaua-Rbs/Deep-Learning-From-A-To-Z-With-Pytorch-And-Python/blob/main/Project_12_CNN_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 12: CNNs: MNIST - Cross-Validation

## Importing Libraries

In [1]:
!pip install skorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 4.0 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from sklearn.model_selection import cross_val_score
import torchvision.transforms as transforms
from skorch import NeuralNetClassifier
torch.__version__

'2.3.1+cu121'

## Database

In [3]:
torch.manual_seed(123)

In [4]:
transform = transforms.ToTensor()

In [5]:
train = datasets.MNIST(root = '.', train = True, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:03<00:00, 3127833.81it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 484608.68it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4227426.01it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4306177.39it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



## Model Structuring

In [6]:
class classificador_torch(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3))
    self.conv2 = nn.Conv2d(32, 32, (3, 3))
    self.activation = nn.ReLU()
    self.bnorm = nn.BatchNorm2d(num_features=32)
    self.pool = nn.MaxPool2d(kernel_size = (2,2))
    self.flatten = nn.Flatten()

    self.linear1 = nn.Linear(in_features=32*5*5, out_features=128)
    self.linear2 = nn.Linear(128, 128)
    self.output = nn.Linear(128, 10)
    self.dropout = nn.Dropout(p = 0.2)

  def forward(self, X):
    X = X.view(-1, 1, 28, 28)
    X = self.pool(self.bnorm(self.activation(self.conv1(X))))
    X = self.pool(self.bnorm(self.activation(self.conv2(X))))
    X = self.flatten(X)

    X = self.dropout(self.activation(self.linear1(X)))
    X = self.dropout(self.activation(self.linear2(X)))
    X = self.output(X)

    return X

In [7]:
classificador_sklearn = NeuralNetClassifier(module = classificador_torch,
                                            criterion = torch.nn.CrossEntropyLoss,
                                            optimizer = torch.optim.Adam,
                                            max_epochs = 3,
                                            batch_size = 10,
                                            train_split = False)

## Cross-Validation

In [8]:
X = train.data.type('torch.FloatTensor')
y = train.targets.type('torch.LongTensor')

In [9]:
resultados = cross_val_score(classificador_sklearn, X, y, cv = 5, scoring = 'accuracy')

  epoch    train_loss      dur
-------  ------------  -------
      1        0.4385  40.9570
      2        0.4014  42.3119
      3        0.4077  43.8496
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4008  40.5295
      2        0.3851  41.4546
      3        0.4037  44.0423
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4190  40.9361
      2        0.4116  41.1821
      3        0.3877  43.7954
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4170  41.5202
      2        0.3737  40.9551
      3        0.3945  41.1603
  epoch    train_loss      dur
-------  ------------  -------
      1        0.4083  44.1695
      2        0.3645  40.8631
      3        0.4004  41.3073


In [10]:
media = resultados.mean()
desvio = resultados.std()

In [11]:
media, desvio

(0.6256333333333333, 0.12603483425007725)

In [12]:
resultados

array([0.62458333, 0.52983333, 0.50033333, 0.6145    , 0.85891667])